In [536]:
import pandas as pd
import numpy as np

---

# Data Prepare

In [537]:
# Set up file path
AsIsPath = r'\\neptune\RAD\2 RuleSets\WIP\2021_Jul_RS_WIP\Score Cut 209 9000S FL_ALI\Exports\SCORECARD_06-30-21_540_AsIs_Base_460_YZH.csv'
ToBePath = r'\\neptune\RAD\2 RuleSets\WIP\2021_Jul_RS_WIP\Score Cut 209 9000S FL_ALI\Exports\SCORECARD_06-30-21_541_ToBe_AllowFL9000s_CarTier_YZH.csv'

In [538]:
# Prepare data for As-is
asis = pd.read_csv(AsIsPath)
asis = asis[['DL_MASTER','BY1_SCORE','BY1_SEG','B1_FACTORTRUST_HIT','CONTRACT_OPENDATE','STA','DEL_STA','%_BOOK','P',"Buyer_1's_State",'BYR1_FICO','BAD_RATE1','VEH_BBCATEGORY','S_DECISION_B1','S_LTVg_B1','S_DISCg_B1']]
asis.rename(columns={"S_DECISION_B1": "AsIsDecision", "S_LTVg_B1": "AsIsATB", "S_DISCg_B1":"AsIsDisc"},inplace=True)
asis['SCOREX'] = asis['BY1_SCORE']/10
asis['SCOREX'] = asis['SCOREX'].apply(np.floor).fillna(0).astype(int)
asis.head()

,DL_MASTER,BY1_SCORE,BY1_SEG,B1_FACTORTRUST_HIT,CONTRACT_OPENDATE,STA,DEL_STA,%_BOOK,P,Buyer_1's_State,BYR1_FICO,BAD_RATE1,VEH_BBCATEGORY,AsIsDecision,AsIsATB,AsIsDisc,SCOREX
0,2954285,209.0,6J,0.0,06-01-21,D,PRE,56.74,I,NV,9002.0,0.3347,MID-SIZE CROSSOVER/SUV,D,0.5966,0.0700,20
1,2956130,225.0,6K,0.0,06-01-21,A,DEL,91.82,I,FL,NaN,0.2696,COMPACT CROSSOVER/SUV,A,1.0149,0.0400,22
2,2961030,237.0,6D,0.0,06-01-21,A,PRE,101.63,I,CA,603.0,0.2815,SPORTY CAR,A,1.1419,0.0200,23
3,2965230,244.0,6G,0.0,06-01-21,A,DEL,108.68,I,CO,608.0,0.2405,MID-SIZE CROSSOVER/SUV,A,1.1049,0.0175,24
4,2966005,238.0,6C,1.0,06-01-21,A,DEL,109.07,I,FL,672.0,0.2895,SPORTY CAR,A,1.0866,0.0360,23


In [539]:
# Check count for As-is data
nasis = asis.shape[0]
asis.shape

(16237, 17)

In [540]:
# Prepare data for To-be
tobe = pd.read_csv(ToBePath)
tobe = tobe[['DL_MASTER','S_TRIGGER1','S_DECISION_B1','S_LTVg_B1','S_DISCg_B1']]
tobe.rename(columns={"S_DECISION_B1": "ToBeDecision", "S_LTVg_B1": "ToBeATB", "S_DISCg_B1":"ToBeDisc", "S_TRIGGER1":"AdjLTV"},inplace=True)
tobe.head()

,DL_MASTER,AdjLTV,ToBeDecision,ToBeATB,ToBeDisc
0,2954285,0.6745,D,0.5966,0.0700
1,2956130,1.0970,A,1.0149,0.0400
2,2961030,1.2108,A,1.1419,0.0200
3,2965230,1.1572,A,1.1049,0.0175
4,2966005,1.1642,A,1.0866,0.0360


In [541]:
# Check count for To-be data
ntobe = tobe.shape[0]
tobe.shape

(16235, 5)

In [542]:
# Join As-is and To-be
df = pd.merge(asis, tobe, on='DL_MASTER')
df.head()

,DL_MASTER,BY1_SCORE,BY1_SEG,B1_FACTORTRUST_HIT,CONTRACT_OPENDATE,STA,DEL_STA,%_BOOK,P,Buyer_1's_State,...,BAD_RATE1,VEH_BBCATEGORY,AsIsDecision,AsIsATB,AsIsDisc,SCOREX,AdjLTV,ToBeDecision,ToBeATB,ToBeDisc
0,2954285,209.0,6J,0.0,06-01-21,D,PRE,56.74,I,NV,...,0.3347,MID-SIZE CROSSOVER/SUV,D,0.5966,0.0700,20,0.6745,D,0.5966,0.0700
1,2956130,225.0,6K,0.0,06-01-21,A,DEL,91.82,I,FL,...,0.2696,COMPACT CROSSOVER/SUV,A,1.0149,0.0400,22,1.0970,A,1.0149,0.0400
2,2961030,237.0,6D,0.0,06-01-21,A,PRE,101.63,I,CA,...,0.2815,SPORTY CAR,A,1.1419,0.0200,23,1.2108,A,1.1419,0.0200
3,2965230,244.0,6G,0.0,06-01-21,A,DEL,108.68,I,CO,...,0.2405,MID-SIZE CROSSOVER/SUV,A,1.1049,0.0175,24,1.1572,A,1.1049,0.0175
4,2966005,238.0,6C,1.0,06-01-21,A,DEL,109.07,I,FL,...,0.2895,SPORTY CAR,A,1.0866,0.0360,23,1.1642,A,1.0866,0.0360


In [543]:
# Check count for joined data
njoin = df.shape[0]
df.shape

(16235, 21)

In [544]:
# Get Date Range
MinDate = min(df['CONTRACT_OPENDATE'])
MaxDate = max(df['CONTRACT_OPENDATE'])
DateRange = 'Deals opened from ' + str(MinDate) + ' to ' + str(MaxDate)

DateRange

'Deals opened from 06-01-21 to 06-30-21'

In [545]:
# Apply filters for joined data
filters = []

filter1 = 'SCOREX is not null'
df = df[df['SCOREX'] != 0]     
filters.append(filter1)

filter2 = 'As-is decision is not null'
df = df[df['AsIsDecision'].notnull()]  
filters.append(filter2)

filter3 = 'To-be decision is not null'
df = df[df['ToBeDecision'].notnull()]    
filters.append(filter3)

df.head()

,DL_MASTER,BY1_SCORE,BY1_SEG,B1_FACTORTRUST_HIT,CONTRACT_OPENDATE,STA,DEL_STA,%_BOOK,P,Buyer_1's_State,...,BAD_RATE1,VEH_BBCATEGORY,AsIsDecision,AsIsATB,AsIsDisc,SCOREX,AdjLTV,ToBeDecision,ToBeATB,ToBeDisc
0,2954285,209.0,6J,0.0,06-01-21,D,PRE,56.74,I,NV,...,0.3347,MID-SIZE CROSSOVER/SUV,D,0.5966,0.0700,20,0.6745,D,0.5966,0.0700
1,2956130,225.0,6K,0.0,06-01-21,A,DEL,91.82,I,FL,...,0.2696,COMPACT CROSSOVER/SUV,A,1.0149,0.0400,22,1.0970,A,1.0149,0.0400
2,2961030,237.0,6D,0.0,06-01-21,A,PRE,101.63,I,CA,...,0.2815,SPORTY CAR,A,1.1419,0.0200,23,1.2108,A,1.1419,0.0200
3,2965230,244.0,6G,0.0,06-01-21,A,DEL,108.68,I,CO,...,0.2405,MID-SIZE CROSSOVER/SUV,A,1.1049,0.0175,24,1.1572,A,1.1049,0.0175
4,2966005,238.0,6C,1.0,06-01-21,A,DEL,109.07,I,FL,...,0.2895,SPORTY CAR,A,1.0866,0.0360,23,1.1642,A,1.0866,0.0360


In [546]:
# Check count for filtered joined data
nfilter = df.shape[0]
df.shape

(15289, 21)

In [547]:
# Write down the filters we used to clean up the joined data
FilterUsed = 'Filters: '
for f in filters: 
    if filters.index(f) != len(filters)-1:
        FilterUsed = FilterUsed + f + ', '
    else: 
        FilterUsed = FilterUsed + f + '.'

FilterUsed

'Filters: SCOREX is not null, As-is decision is not null, To-be decision is not null.'

In [548]:
# Track the count of data cleaning process
AsIsCount = 'As-is Count: ' + str(nasis) 
ToBeCount = 'To-be Count: ' + str(ntobe) 
JoinedCount = 'Joined Count: ' + str(njoin) 
FilteredCount = 'Filtered Count: ' + str(nfilter)

print(AsIsCount)
print(ToBeCount)
print(JoinedCount)
print(FilteredCount)

As-is Count: 16237
To-be Count: 16235
Joined Count: 16235
Filtered Count: 15289


---

# Analysis

In [549]:
# Set up Subpopulation Group
GroupName = "Buyer_1's_State"

In [550]:
# Function of table formatting
def FormatCnt(df):
    df = df.fillna(0).astype(int)
    df.columns = df.columns.droplevel()
    df.rename(columns={"All": "Total"},index = {"All": "Total"} ,inplace=True)
    df.insert(0, 'Total', df.pop("Total"))
    return df

In [551]:
# Decision Change Count
DecisionChange = pd.pivot_table(df, values=['DL_MASTER'], index=['AsIsDecision'], columns=['ToBeDecision'], aggfunc=len,margins=True)
DecisionChange = FormatCnt(DecisionChange)
DecisionChange.name = 'Decision Change'
DecisionChange

ToBeDecision,Total,A,C,D
AsIsDecision,,,,
A,8610,8606,4,0
C,1425,0,1425,0
D,5254,60,41,5153
Total,15289,8666,1470,5153


In [552]:
# Prep data for Approved to Approved population
dfapp = df[(df.AsIsDecision=='A') & (df.ToBeDecision=='A')]
dfapp.shape

(8606, 21)

In [553]:
# Approved to Approved Count
ApprovedToApprovedCount = pd.pivot_table(dfapp, values=['DL_MASTER'], index=[GroupName], columns=['SCOREX'], aggfunc=len,margins=True)
ApprovedToApprovedCount = FormatCnt(ApprovedToApprovedCount)
ApprovedToApprovedCount.name = 'Approved to Approved Count'
ApprovedToApprovedCount

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
Buyer_1's_State,,,,,,,,,,,,,
AZ,421,0,0,35,100,92,71,58,36,14,13,2,0
CA,3535,0,0,269,789,825,511,394,340,234,114,37,22
CO,228,0,0,9,59,63,38,30,18,8,2,1,0
FL,2217,0,0,0,102,1493,397,156,63,4,2,0,0
ID,3,0,0,0,2,1,0,0,0,0,0,0,0
NM,103,0,0,9,27,18,15,17,10,3,4,0,0
NV,273,0,0,41,64,66,35,32,22,8,4,1,0
OR,132,1,0,0,34,35,17,14,16,6,6,3,0
TX,1515,0,1,0,525,486,271,132,65,31,4,0,0


In [554]:
# As-is ATB
AsIsAtb = pd.pivot_table(dfapp, values=['AsIsATB'], index=[GroupName], columns=['SCOREX'], aggfunc=np.mean,margins=True)
AsIsAtb = AsIsAtb*100
AsIsAtb = AsIsAtb.fillna(0)
AsIsAtb = AsIsAtb.round(1)
AsIsAtb.columns = AsIsAtb.columns.droplevel()
AsIsAtb.rename(columns={"All": "Total"},index = {"All": "Total"} ,inplace=True)
AsIsAtb.insert(0, 'Total', AsIsAtb.pop("Total"))
AsIsAtb.replace(0.0, np.nan, inplace=True)
AsIsAtb.name = 'As Is ATB'
AsIsAtb

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
Buyer_1's_State,,,,,,,,,,,,,
AZ,99.3,NaN,NaN,93.7,93.4,97.8,101.7,105.1,104.5,108.8,105.4,112.5,NaN
CA,97.8,NaN,NaN,92.3,93.4,96.7,100.4,102.3,102.8,101.6,98.6,96.9,93.3
CO,96.2,NaN,NaN,94.4,94.5,92.7,94.4,102.4,102.9,106.9,116.7,76.6,NaN
FL,98.4,NaN,NaN,NaN,93.0,97.5,100.6,102.1,103.9,111.0,115.1,NaN,NaN
ID,79.0,NaN,NaN,NaN,83.9,69.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NM,95.8,NaN,NaN,96.0,85.8,90.4,100.0,101.8,105.3,114.6,109.1,NaN,NaN
NV,100.2,NaN,NaN,94.4,95.6,100.1,106.0,106.6,99.6,115.1,107.0,95.5,NaN
OR,92.1,98.6,NaN,NaN,90.1,88.4,98.8,89.1,94.4,105.5,91.9,94.2,NaN
TX,94.0,NaN,105.8,NaN,89.5,93.4,98.9,100.4,100.7,92.1,113.0,NaN,NaN


In [555]:
# To-be ATB
ToBeAtb = pd.pivot_table(dfapp, values=['ToBeATB'], index=[GroupName], columns=['SCOREX'], aggfunc=np.mean,margins=True)
ToBeAtb = ToBeAtb*100
ToBeAtb = ToBeAtb.fillna(0)
ToBeAtb = ToBeAtb.round(1)
ToBeAtb.columns = ToBeAtb.columns.droplevel()
ToBeAtb.rename(columns={"All": "Total"},index = {"All": "Total"} ,inplace=True)
ToBeAtb.insert(0, 'Total', ToBeAtb.pop("Total"))
ToBeAtb.replace(0.0, np.nan, inplace=True)
ToBeAtb.name = 'To Be ATB'
ToBeAtb

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
Buyer_1's_State,,,,,,,,,,,,,
AZ,99.3,NaN,NaN,93.7,93.4,97.8,101.7,105.1,104.5,108.8,105.4,112.5,NaN
CA,97.8,NaN,NaN,92.3,93.4,96.7,100.4,102.3,102.8,101.6,98.6,96.9,93.3
CO,96.2,NaN,NaN,94.4,94.5,92.7,94.4,102.4,102.9,106.9,116.7,76.6,NaN
FL,98.4,NaN,NaN,NaN,93.0,97.5,100.6,102.1,103.9,111.0,115.1,NaN,NaN
ID,79.0,NaN,NaN,NaN,83.9,69.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NM,95.8,NaN,NaN,96.0,85.8,90.4,100.0,101.8,105.3,114.6,109.1,NaN,NaN
NV,100.2,NaN,NaN,94.4,95.6,100.1,106.0,106.6,99.6,115.1,107.0,95.5,NaN
OR,92.1,98.6,NaN,NaN,90.1,88.4,98.8,89.1,94.4,105.5,91.9,94.2,NaN
TX,94.0,NaN,105.8,NaN,89.5,93.4,98.9,100.4,100.7,92.1,113.0,NaN,NaN


In [556]:
# As-is Disc
AsIsDisc = pd.pivot_table(dfapp, values=['AsIsDisc'], index=[GroupName], columns=['SCOREX'], aggfunc=np.mean,margins=True)
AsIsDisc = AsIsDisc*100
AsIsDisc = AsIsDisc.fillna(0)
AsIsDisc = AsIsDisc.round(1)
AsIsDisc.columns = AsIsDisc.columns.droplevel()
AsIsDisc.rename(columns={"All": "Total"},index = {"All": "Total"} ,inplace=True)
AsIsDisc.insert(0, 'Total', AsIsDisc.pop("Total"))
AsIsDisc.replace(0.0, np.nan, inplace=True)
AsIsDisc.name = 'As Is Disc'
AsIsDisc

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
Buyer_1's_State,,,,,,,,,,,,,
AZ,5.5,NaN,NaN,7.9,7.9,5.8,3.9,4.2,2.4,3.7,3.2,1.3,NaN
CA,4.5,NaN,NaN,7.4,7.0,4.6,3.5,3.4,2.5,2.3,2.3,2.0,1.7
CO,5.1,NaN,NaN,7.3,6.6,5.4,4.2,3.3,2.7,3.9,8.2,6.6,NaN
FL,4.2,NaN,NaN,NaN,7.1,4.5,3.2,2.8,2.4,4.7,8.5,NaN,NaN
ID,5.3,NaN,NaN,NaN,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NM,5.6,NaN,NaN,7.2,7.9,5.9,3.7,4.5,3.7,4.6,1.7,NaN,NaN
NV,5.7,NaN,NaN,8.5,7.4,5.3,4.2,3.5,3.9,5.3,1.8,1.0,NaN
OR,3.4,6.0,NaN,NaN,6.1,4.0,2.3,1.0,1.1,1.6,1.8,2.7,NaN
TX,5.6,NaN,6.5,NaN,7.9,5.1,3.7,3.6,3.8,1.9,4.6,NaN,NaN


In [557]:
# To-be Disc
ToBeDisc = pd.pivot_table(dfapp, values=['ToBeDisc'], index=[GroupName], columns=['SCOREX'], aggfunc=np.mean,margins=True)
ToBeDisc = ToBeDisc*100
ToBeDisc = ToBeDisc.fillna(0)
ToBeDisc = ToBeDisc.round(1)
ToBeDisc.columns = ToBeDisc.columns.droplevel()
ToBeDisc.rename(columns={"All": "Total"},index = {"All": "Total"} ,inplace=True)
ToBeDisc.insert(0, 'Total', ToBeDisc.pop("Total"))
ToBeDisc.replace(0.0, np.nan, inplace=True)
ToBeDisc.name = 'To Be Disc'
ToBeDisc

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
Buyer_1's_State,,,,,,,,,,,,,
AZ,5.5,NaN,NaN,7.9,7.9,5.8,3.9,4.2,2.4,3.7,3.2,1.3,NaN
CA,4.5,NaN,NaN,7.4,7.0,4.6,3.5,3.4,2.5,2.3,2.3,2.0,1.7
CO,5.1,NaN,NaN,7.3,6.6,5.4,4.2,3.3,2.7,3.9,8.2,6.6,NaN
FL,4.2,NaN,NaN,NaN,7.1,4.5,3.2,2.8,2.4,4.7,8.5,NaN,NaN
ID,5.3,NaN,NaN,NaN,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NM,5.6,NaN,NaN,7.2,7.9,5.9,3.7,4.5,3.7,4.6,1.7,NaN,NaN
NV,5.7,NaN,NaN,8.5,7.4,5.3,4.2,3.5,3.9,5.3,1.8,1.0,NaN
OR,3.4,6.0,NaN,NaN,6.1,4.0,2.3,1.0,1.1,1.6,1.8,2.7,NaN
TX,5.6,NaN,6.5,NaN,7.9,5.1,3.7,3.6,3.8,1.9,4.6,NaN,NaN


In [558]:
# ATB Diff
AtbDiff = ToBeAtb.fillna(0)-AsIsAtb.fillna(0)
AtbDiff.name = 'ATB Diff'
AtbDiff

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
Buyer_1's_State,,,,,,,,,,,,,
AZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [559]:
# Disc Diff
DiscDiff = ToBeDisc.fillna(0)-AsIsDisc.fillna(0)
DiscDiff.name = 'Disc Diff'
DiscDiff

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
Buyer_1's_State,,,,,,,,,,,,,
AZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---

# Approval Volume Change

In [560]:
# Prep As-is Approved and To-be Approved data
dfasisapp = df[df.AsIsDecision=='A']
dftobeapp = df[df.ToBeDecision=='A']

# Calculate the count of As-is Approved and To-be Approved population
N_AsIsA = len(dfasisapp)
N_ToBeA = len(dftobeapp)

# Initiate a list to save tables if there's any approval volume change 
VolChgLst = []

# Check if approval volume changes. If yes, append the analysis tables to the list created above
if N_AsIsA != N_ToBeA:
    # As-is Approved Count
    AsIsApprovedCount = pd.pivot_table(dfasisapp, values=['DL_MASTER'], index=[GroupName], columns=['SCOREX'], aggfunc=len,margins=True)
    AsIsApprovedCount = FormatCnt(AsIsApprovedCount)
    AsIsApprovedCount.name = 'As Is Approved Count'
    VolChgLst.append(AsIsApprovedCount)
    
    # To-be Approved Count
    ToBeApprovedCount = pd.pivot_table(dftobeapp, values=['DL_MASTER'], index=[GroupName], columns=['SCOREX'], aggfunc=len,margins=True)
    ToBeApprovedCount = FormatCnt(ToBeApprovedCount)
    ToBeApprovedCount.name = 'To Be Approved Count'
    VolChgLst.append(ToBeApprovedCount)
    
    # Volume Change Count
    VolumeChange = ToBeApprovedCount - AsIsApprovedCount
    VolumeChange.name = 'Volume Change Count'
    VolChgLst.append(VolumeChange)
    
    # Volume Change in %
    VolumeChangePct = 100*(ToBeApprovedCount - AsIsApprovedCount)/AsIsApprovedCount
    VolumeChangePct = VolumeChangePct.round(1)
    VolumeChangePct.name = 'Volume Change in %'
    VolChgLst.append(VolumeChangePct)

In [561]:
AsIsApprovedCount

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
Buyer_1's_State,,,,,,,,,,,,,
AZ,421,0,0,35,100,92,71,58,36,14,13,2,0
CA,3537,0,0,269,790,826,511,394,340,234,114,37,22
CO,228,0,0,9,59,63,38,30,18,8,2,1,0
FL,2218,0,0,0,102,1493,398,156,63,4,2,0,0
ID,3,0,0,0,2,1,0,0,0,0,0,0,0
NM,103,0,0,9,27,18,15,17,10,3,4,0,0
NV,273,0,0,41,64,66,35,32,22,8,4,1,0
OR,132,1,0,0,34,35,17,14,16,6,6,3,0
TX,1516,0,1,0,525,487,271,132,65,31,4,0,0


In [562]:
ToBeApprovedCount

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
Buyer_1's_State,,,,,,,,,,,,,
AZ,421,0,0,35,100,92,71,58,36,14,13,2,0
CA,3535,0,0,269,789,825,511,394,340,234,114,37,22
CO,228,0,0,9,59,63,38,30,18,8,2,1,0
FL,2277,0,0,0,162,1493,397,156,63,4,2,0,0
ID,3,0,0,0,2,1,0,0,0,0,0,0,0
NM,103,0,0,9,27,18,15,17,10,3,4,0,0
NV,273,0,0,41,64,66,35,32,22,8,4,1,0
OR,132,1,0,0,34,35,17,14,16,6,6,3,0
TX,1515,0,1,0,525,486,271,132,65,31,4,0,0


In [563]:
VolumeChange

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
Buyer_1's_State,,,,,,,,,,,,,
AZ,0,0,0,0,0,0,0,0,0,0,0,0,0
CA,-2,0,0,0,-1,-1,0,0,0,0,0,0,0
CO,0,0,0,0,0,0,0,0,0,0,0,0,0
FL,59,0,0,0,60,0,-1,0,0,0,0,0,0
ID,0,0,0,0,0,0,0,0,0,0,0,0,0
NM,0,0,0,0,0,0,0,0,0,0,0,0,0
NV,0,0,0,0,0,0,0,0,0,0,0,0,0
OR,0,0,0,0,0,0,0,0,0,0,0,0,0
TX,-1,0,0,0,0,-1,0,0,0,0,0,0,0


In [564]:
VolumeChangePct

SCOREX,Total,18,19,20,21,22,23,24,25,26,27,28,29
Buyer_1's_State,,,,,,,,,,,,,
AZ,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
CA,-0.1,NaN,NaN,0.0,-0.1,-0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CO,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
FL,2.7,NaN,NaN,NaN,58.8,0.0,-0.3,0.0,0.0,0.0,0.0,NaN,NaN
ID,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NM,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
NV,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
OR,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
TX,-0.1,NaN,0.0,NaN,0.0,-0.2,0.0,0.0,0.0,0.0,0.0,NaN,NaN


---

# Decision and Pricing Changes

In [565]:
# Prepare datasets to review if there're any deals with decision changes or pricing changes.

# Initiate a list 
ChgList = []

In [566]:
# Decision Changes

# If there's any decision change, put the dataset into the list created above
for i in ['A','C','D']:
    for j in ['A','C','D']:
        if i != j:
            dfdecchg = df[(df.AsIsDecision==i) & (df.ToBeDecision==j)]
            dfdecchg.name = i + ' to ' + j
            count = len(dfdecchg)
            if count > 0:
                ChgList.append(dfdecchg)

In [567]:
# Pricing Changes

# If there's any ATB change, put the dataset into the list created above
df_atb_chg = dfapp[dfapp.AsIsATB != dfapp.ToBeATB]
df_atb_chg.name = 'ATB Change'
if len(df_atb_chg) > 0:
    ChgList.append(df_atb_chg)

# If there's any Disc change, put the dataset into the list created above
df_disc_chg = dfapp[dfapp.AsIsDisc != dfapp.ToBeDisc]
df_disc_chg.name = 'Disc Change'
if len(df_disc_chg) > 0:
    ChgList.append(df_disc_chg)

# Export the Result to Excel File

In [568]:
# Set up tables to put into Excel File

# list of dataframes to put in Analysis Tab
dfs = [DecisionChange, ApprovedToApprovedCount, AsIsAtb, ToBeAtb, AsIsDisc, ToBeDisc, AtbDiff, DiscDiff]
dfs.extend(VolChgLst)

In [569]:
# Funtion of Excel Export
def multiple_dfs(df_list, sheets, file_name, spaces, DecisionChgList=[]):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter') 

    row = 8
    for dataframe in df_list:
        dataframe.to_excel(writer,sheet_name=sheets,startrow=row , startcol=1) 
        writer.sheets[sheets].write(row, 0, dataframe.name)
        row = row + len(dataframe.index) + spaces + 1
    writer.sheets[sheets].write(0, 0, DateRange)
    writer.sheets[sheets].write(1, 0, FilterUsed)
    writer.sheets[sheets].write(2, 0, 'Count Tracking')
    writer.sheets[sheets].write(2, 1, AsIsCount)
    writer.sheets[sheets].write(3, 1, ToBeCount)
    writer.sheets[sheets].write(4, 1, JoinedCount)
    writer.sheets[sheets].write(5, 1, FilteredCount)
    writer.sheets[sheets].set_column(0,0,30)
    writer.sheets[sheets].set_column(1,1,20)
    
    if len(DecisionChgList) > 0:
        for df in DecisionChgList:
            df.to_excel(writer, sheet_name=df.name, index=False)
    writer.save()

# Run function
multiple_dfs(dfs, 'Analysis', 'Analysis1_YZH.xlsx', 1, ChgList)